# General remarks and introduction

This document serves to demonstrate the simulation functions of the `sbmfi` (simulation based metabolic flux inference) `python` package. The folder structure of the package is:
```
sbmfi
|___ core
|___ inference
|___ models
|___ lcmsanalysis
|___ priors
```
The `core` folder contains all the algorithms that are necessary to specify a simulator that takes as input fluxes and outputs mass distribution vectors (MDV) of metabolites in a steady-state $^{13}C$ carbon labelling experiment (CLE) using the [EMU algorithm](https://www.sciencedirect.com/science/article/pii/S109671760600084X?via%3Dihub). It also contains a dataset simulator to simulate measurement. The `inference` folder contains a uniform prior over the flux polytope. The `models` folder is self-explanatory. Last, the `lcmsanalysis` contains the `formula` file that specifies a convenient class for chemical formulas.


### LinAlg

An object that we will see throughout the package is instatiations of `LinAlg`. We originally developed the package working with `numpy`, but soon found that having `torch` as a back-end for all numerics would be very useful due to GPU support and automatic differentiation. Since the two packages have slightly different API, we decided to create a unified API which is `LinAlg`. 

In [2]:
from sbmfi.core.linalg import LinAlg

In [4]:
la = LinAlg(
    backend = 'torch',  # torch or numpy backend
    batch_size = 2,  # in all labelling simulation algorithms, we simulate in batches; this parameter specifies the batch_size
    solver  = 'lu_solve_ex',  # for torch there are 2 
    device = 'cpu',
    fkwargs = None,
    seed = 42,
    dtype = np.float32,
)

### Models, Reactions and Metabolites

The package has been developed based on the structure of `cobrapy`([link](https://cobrapy.readthedocs.io/en/latest/index.html)). The intention is that anyone working with `cobra` can use `sbmfi` quickly. The general phylosophy on the relation between the two packages is that one should build a model and perform various flux-analyses using `cobra` and once a model is 'finished', it can be equipped with labelling information and used to sample fluxes and simulate labelling in `sbmfi`. 

For these reasons, `sbmfi.core` has modules defining the `LabelledMetabolite(cobra.Metabolite)`, `LabellingReaction(cobra.Reaction)` and `LabellingModel(cobra.Model)` classes which inherit functionality from their `cobra` classes. Each of these `Labelling` classes is instantiated with a `cobra` object

In [25]:
from sbmfi.core.model import model_builder_from_dict, EMU_Model
reaction_kwargs = {
    'a_in': {
        'upper_bound': 10.0, 'lower_bound': 10.0,
        'atom_map_str': '∅ --> A/abc'
    },
    'co2_out': {
        'upper_bound': 100.0,
        'atom_map_str': 'co2/a --> ∅'
    },
    'e_out': {
        'upper_bound': 100.0,
        'atom_map_str': 'E/ab --> ∅'
    },
    'v1': {
        'upper_bound': 100.0,
        'atom_map_str': 'A/abc --> B/ab + D/c'
    },
    'v2': {
        'upper_bound': 100.0,
        'atom_map_str': 'A/abc --> C/bc + D/a'
    },
    'v3': {
        'upper_bound': 100.0,
        'atom_map_str': 'B/ab + D/c --> E/ac + co2/b'
    },
    'v4': {
        'upper_bound': 100.0,
        'atom_map_str': 'C/ab + D/c --> E/cb + co2/a'
    },
}
metabolite_kwargs = {
    'E': {'formula': 'C2H4O2'},
    'D': {'formula': 'CH4'},
}

cobra_model = model_builder_from_dict(metabolite_kwargs=metabolite_kwargs, reaction_kwargs=reaction_kwargs)
print(f'type of model: {type(cobra_model)}')

model = EMU_Model(linalg=la, model=cobra_model)
model.add_labelling_kwargs(metabolite_kwargs=metabolite_kwargs, reaction_kwargs=reaction_kwargs)
for reaction in model.reactions:
    print(reaction, reaction.bounds)

type of model: <class 'cobra.core.model.Model'>
a_in:  --> A/abc (10.0, 10.0)
co2_out: co2/a -->  (0.0, 100.0)
e_out: E/ab -->  (0.0, 100.0)
v1: A/abc --> B/ab + D/c (0.0, 100.0)
v2: A/abc --> C/bc + D/a (0.0, 100.0)
v3: B/ab + D/c --> E/ac + co2/b (0.0, 100.0)
v4: C/ab + D/c --> E/cb + co2/a (0.0, 100.0)


In [ ]:
model